# Data Mining

1. Display the collected data.

2. Deal with data and make it the csv file we want.
   1. 1 second time window


In [70]:
import pandas as pd
import numpy as np
from datetime import datetime

In [44]:
cheatData = []
for ad in ['../raw_data/user_1_1.log', '../raw_data/user_2_1.log']:#, '../data/normal_user1_1.log']:
    f = open(ad)
    cheatData+=f.readlines()

## Display the raw data


In [45]:
print(len(cheatData))
cheatData[:5]

17299


['label: 0, timestamp: 1650243539, dist_s: 628.077, angle_s: 230.351, mean_dist: 5764.6, min_dist: 5094.55, view_dist: 74.2955, key_2: 1, shooting: 0, aiming: 0, moving: 0, average_mouse_speed: 0\n',
 'label: 0, timestamp: 1650243540, dist_s: 984.68, angle_s: 0, mean_dist: -nan, min_dist: 1e+07, view_dist: 74.2955, key_45: 1, key_121: 2, shooting: 0, aiming: 0, moving: 0, average_mouse_speed: 0\n',
 'label: 0, timestamp: 1650243541, dist_s: 1247.55, angle_s: 29.7836, mean_dist: 5807.95, min_dist: 5296.89, view_dist: 64.6675, key_45: 1, shooting: 0, aiming: 0, moving: 0, average_mouse_speed: 0\n',
 'label: 0, timestamp: 1650243542, dist_s: 1300.78, angle_s: 28.9494, mean_dist: 5955.68, min_dist: 5187.03, view_dist: 81.906, shooting: 0, aiming: 0, moving: 0, average_mouse_speed: 12\n',
 'label: 0, timestamp: 1650243543, dist_s: 1363.45, angle_s: 3.18005, mean_dist: 7257.5, min_dist: 7221.94, view_dist: 80.5923, shooting: 0, aiming: 0, moving: 0, average_mouse_speed: 4\n']

## Data Preprocess


In [46]:
# define some consts
useful_key_set = [87, 65, 83, 68, 81, 69, 162, 9, 70, 71, 49, 50, 51, 52]

In [47]:
# define useful and useless key press function
def useful_key_press(keyname, value, ret):
    keynum = int(keyname[4:])
    presstime = int(value)
    if keynum in useful_key_set:
        ret['useful'] += presstime
    else:
        ret['useless'] += presstime

# for each data define a function to process it, return a dictionary
def convert_line_to_dict(record:str, keymode:str='useful')->dict:
    ret = dict()

    # default value for useful key
    if keymode == 'useful':
        ret['useful'] = 0
        ret['useless'] = 0

    features = record.split(',')
    for f in features:
        [keyname, value] = f.split(': ')
        keyname = keyname[1:] if keyname[0] == ' ' else keyname
        # timestamp
        if keyname=='timestamp':
            ret['timestamp'] = datetime.fromtimestamp(int(value))
            continue
        # average_mouse_speed
        if keyname=='average_mouse_speed':
            ret['average_mouse_speed'] = int(value)
            continue
        # dist_s
        if keyname=='dist_s':
            ret['dist_s'] = float(value)
            continue
        # angle_s
        if keyname =='angle_s':
            ret['angle_s'] = float(value)
            continue
        # mean_dist
        if keyname == 'mean_dist':
            if value == '-nan':
                ret['mean_dist'] = None
            else:
                ret['mean_dist'] = float(value)
            continue
        # min_dist
        if keyname == 'min_dist':
            if value == '1e+07':
                ret['min_dist'] = None
            else:
                ret['min_dist'] = float(value)
            continue
        # view_dist
        if keyname == 'view_dist':
            if value == '-1':
                ret['view_dist'] = None
            else:
                ret['view_dist'] = float(value)
            continue
        # shooting
        if keyname == 'shooting':
            ret['shooting'] = int(value)
            continue
        # aiming
        if keyname == 'aiming':
            ret['aiming'] = int(value)
            continue
        # moving
        if keyname == 'moving':
            ret['moving'] = int(value)
            continue
        # key input
        if keyname[:3]=='key':
            # depends on how you handle the key
        
            # use useful and useless key record
            if keymode=='useful':
                useful_key_press(keyname, value, ret)
                continue
        # label
        if keyname=='label':
            ret['class'] = int(value)
    # if not labeled then its normal data
    # if 'class' not in ret:
    #     ret['class'] = 0
    return ret

def convert_dicts_to_dataframe(dicts, keymode:str='useful'):
    labels = list(dicts[0].keys())
    arrs = [list(d.values()) for d in dicts]
    df = pd.DataFrame(arrs, columns = labels)
    if keymode == 'useful':
        df = df[['timestamp', 'dist_s', 'angle_s', 'mean_dist', \
            'min_dist', 'view_dist', 'average_mouse_speed', \
            'useful', 'useless', 'aiming', 'shooting', 'moving', 'class']]
    return df


In [48]:
dicts = [convert_line_to_dict(record) for record in cheatData]
df = convert_dicts_to_dataframe(dicts)
df

,timestamp,dist_s,angle_s,mean_dist,min_dist,view_dist,average_mouse_speed,useful,useless,aiming,shooting,moving,class
0,2022-04-17 20:58:59,628.077,230.35100,5764.60,5094.55,74.2955,0,0,1,0,0,0,0
1,2022-04-17 20:59:00,984.680,0.00000,NaN,NaN,74.2955,0,0,3,0,0,0,0
2,2022-04-17 20:59:01,1247.550,29.78360,5807.95,5296.89,64.6675,0,0,1,0,0,0,0
3,2022-04-17 20:59:02,1300.780,28.94940,5955.68,5187.03,81.9060,12,0,0,0,0,0,0
4,2022-04-17 20:59:03,1363.450,3.18005,7257.50,7221.94,80.5923,4,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17294,2022-04-18 02:47:57,1200.150,0.00000,2560.58,2416.79,83.2115,0,0,0,0,0,0,0
17295,2022-04-18 02:47:58,1266.950,0.00000,2746.02,2625.07,83.2115,0,0,0,0,0,0,0
17296,2022-04-18 02:47:59,1221.210,0.00000,2996.24,2868.48,83.2115,0,0,1,0,0,0,0
17297,2022-04-18 02:48:00,1228.330,0.00000,3231.47,3080.77,83.2115,0,0,1,0,0,0,0


In [49]:
path = '../csv_data/user_1.csv'
df.to_csv(path, index=False)

In [50]:
df = pd.read_csv(path)

In [114]:
# seperate time window.
timewindow = 10

def createDfTimeWindow(df, timewindow):
    arrWithTimeWindow = []
    for i in range(0, len(df)-timewindow):
        slc = df[i:i+timewindow]
        arr = slc.to_numpy()
        t = datetime.fromisoformat(arr[-1][0])-datetime.fromisoformat(arr[0][0])
        if t.seconds!=timewindow-1:
            continue
        newl = []
        # prepare timeline
        newl.append(arr[0][0])
        # dist_s
        newl.append(sum(arr.T[1]))
        # angle_s
        newl.append(sum(arr.T[2]))
        # mean_dist
        t = arr.T[3]
        t = [i for i in t if pd.notna(i)]
        newl.append(sum(t)/len(t) if len(t) else np.nan)
        # min_dist
        t = arr.T[4]
        t = [i for i in t if pd.notna(i)]
        newl.append(min(t) if len(t) else np.nan)
        # view_dist
        t = arr.T[5]
        t = [i for i in t if pd.notna(i)]
        newl.append(sum(t)/len(t) if len(t) else np.nan)
        # average_mouse_speed
        t = arr.T[6]
        newl.append(sum(t)/len(t))
        # useful
        newl.append(sum(arr.T[7]))
        # useless
        newl.append(sum(arr.T[8]))
        # aiming
        newl.append(sum(arr.T[9]))
        # shooting
        newl.append(sum(arr.T[10]))
        # moving
        newl.append(sum(arr.T[11]))
        # class
        newl.append(max(arr.T[12]))

        arrWithTimeWindow.append(newl)

    columns = ['timestamp', 'dist_s', 'angle_s', 'mean_dist', \
                'min_dist', 'view_dist', 'average_mouse_speed', \
                'useful', 'useless', 'aiming', 'shooting', 'moving', 'class']

    newdf = pd.DataFrame(arrWithTimeWindow, columns=columns)
    return newdf

newdf = createDfTimeWindow(df, timewindow)
newdf

,timestamp,dist_s,angle_s,mean_dist,min_dist,view_dist,average_mouse_speed,useful,useless,aiming,shooting,moving,class
0,2022-04-17 20:58:59,12234.817,851.264350,5926.058889,4293.470,66.360005,14.4,0,6,0,0,0,0
1,2022-04-17 20:59:00,12921.470,630.813630,5766.480000,4229.800,58.931064,14.4,0,5,0,0,0,0
2,2022-04-17 20:59:01,13246.160,633.257490,5617.644000,4191.120,51.505652,14.4,0,2,0,0,0,0
3,2022-04-17 20:59:02,13363.560,606.751820,5456.404000,3999.740,45.041425,14.4,0,1,0,0,0,0
4,2022-04-17 20:59:03,13455.260,587.234200,5282.626000,3633.000,36.861430,13.2,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17185,2022-04-18 02:47:47,15528.310,151.689249,1737.436167,766.278,85.998414,3.2,0,0,0,0,0,0
17186,2022-04-18 02:47:48,14759.380,151.689249,1855.028143,766.278,85.650050,3.2,0,0,0,0,0,0
17187,2022-04-18 02:47:49,14001.000,151.689249,1966.402125,766.278,85.379100,3.2,0,0,0,0,0,0
17188,2022-04-18 02:47:50,13244.580,151.689249,2080.828556,766.278,85.162340,3.2,0,1,0,0,0,0


In [116]:
path = '../csv_data/user_1_timewindow'+str(timewindow)+'.csv'
newdf.to_csv(path, index=False)